In [8]:
import cv2
import numpy as np

In [9]:
'''
感知哈希算法是一个比均值哈希算法更为健壮的一种算法，与均值哈希算法的区别在于感知哈希算法是通过DCT（离散余弦变换）来获取图片的低频信息。
离散余弦变换（DCT）是种图像压缩算法，它将图像从像素域变换到频率域。然后一般图像都存在很多冗余和相关性的，所以转换到频率域之后，只有很少的一部分频率分量的系数才不为0，大部分系数都为0（或者说接近于0）。经过DCT变换后的系数矩阵从左上角到右下角频率越来越高，因此图片的能量主要保留在左上角的低频系数上了。
具体步骤：
（1）缩小尺寸：pHash以小图片开始，但图片大于8x8，32x32是最好的。这样做的目的是简化了DCT的计算，而不是减小频率。
（2）简化色彩：将图片转化成灰度图像，进一步简化计算量。
（3）计算DCT：计算图片的DCT变换，得到32x32的DCT系数矩阵。
（4）缩小DCT：虽然DCT的结果是32x32大小的矩阵，但我们只要保留左上角的8x8的矩阵，这部分呈现了图片中的最低频率。
（5）计算平均值：如同均值哈希一样，计算DCT的均值。
（6）计算hash值：这是最主要的一步，根据8x8的DCT矩阵，设置0或1的64位的hash值，大于等于DCT均值的设为”1”，小于DCT均值的设为“0”。组合在一起，就构成了一个64位的整数，这就是这张图片的指纹。
分析： 结果并不能告诉我们真实性的低频率，只能粗略地告诉我们相对于平均值频率的相对比例。只要图片的整体结构保持不变，hash结果值就不变。能够避免伽马校正或颜色直方图被调整带来的影响。对于变形程度在25%以内的图片也能精准识别。
'''

#感知哈希算法
def pHash(image): 
    image = cv2.resize(image,(32,32), interpolation=cv2.INTER_CUBIC) 
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
#     cv2.imshow('image', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    # 将灰度图转为浮点型，再进行dct变换 
    dct = cv2.dct(np.float32(image))
#     print(dct)
    # 取左上角的8*8，这些代表图片的最低频率 
    # 这个操作等价于c++中利用opencv实现的掩码操作 
    # 在python中进行掩码操作，可以直接这样取出图像矩阵的某一部分 
    dct_roi = dct[0:8,0:8]  
    avreage = np.mean(dct_roi) 
    hash = [] 
    for i in range(dct_roi.shape[0]): 
        for j in range(dct_roi.shape[1]): 
            if dct_roi[i,j] > avreage: 
                hash.append(1) 
            else: 
                hash.append(0) 
    return hash

In [10]:
'''
一张图片就是一个二维信号，它包含了不同频率的成分。亮度变化小的区域是低频成分，它描述大范围的信息。而亮度变化剧烈的区域（比如物体的边缘）就是高频的成分，它描述具体的细节。或者说高频可以提供图片详细的信息，而低频可以提供一个框架。 而一张大的，详细的图片有很高的频率，而小图片缺乏图像细节，所以都是低频的。所以我们平时的下采样，也就是缩小图片的过程，实际上是损失高频信息的过程。均值哈希算法就是利用图片的低频信息。
具体步骤：
（1）缩小尺寸：将图片缩小到8x8的尺寸，总共64个像素。这一步的作用是去除图片的细节，只保留结构、明暗等基本信息，摒弃不同尺寸、比例带来的图片差异。
（2）简化色彩：将缩小后的图片，转为64级灰度。也就是说，所有像素点总共只有64种颜色。
（3）计算平均值：计算所有64个像素的灰度平均值
（4）比较像素的灰度：将每个像素的灰度，与平均值进行比较。大于或等于平均值，记为1；小于平均值，记为0。
（5）计算哈希值：将上一步的比较结果，组合在一起，就构成了一个64位的整数，这就是这张图片的指纹。组合的次序并不重要，只要保证所有图片都采用同样次序就行了。
最后得到两张图片的指纹信息后，计算两组64位数据的汉明距离，即对比数据不同的位数，不同位数越少，表明图片的相似度越大。
分析： 均值哈希算法计算速度快，不受图片尺寸大小的影响，但是缺点就是对均值敏感，例如对图像进行伽马校正或直方图均衡就会影响均值，从而影响最终的hash值。
'''

#均值哈希算法
def aHash(image):
    #缩放为8*8
    image=cv2.resize(image,(8,8),interpolation=cv2.INTER_CUBIC)
    #转换为灰度图
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    avreage = np.mean(image) 
    hash = [] 
    for i in range(image.shape[0]): 
        for j in range(image.shape[1]): 
            if image[i,j] > avreage: 
                hash.append(1) 
            else: 
                hash.append(0) 
    return hash

In [11]:
'''
比pHash，dHash的速度要快的多，相比aHash，dHash在效率几乎相同的情况下的效果要更好，它是基于渐变实现的。
主要步骤：
（1）缩小尺寸：收缩到8x9（高x宽）的大小，一遍它有72的像素点
（2）转化为灰度图：把缩放后的图片转化为256阶的灰度图。
（3）计算差异值：dHash算法工作在相邻像素之间，这样每行9个像素之间产生了8个不同的差异，一共8行，则产生了64个差异值
（4）获得指纹：如果左边的像素比右边的更亮，则记录为1，否则为0.
'''

#差值感知算法
def dHash(image):
    #缩放9*8
    image=cv2.resize(image,(9,8),interpolation=cv2.INTER_CUBIC)
    #转换灰度图
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#     print(image.shape)
    hash=[]
    #每行前一个像素大于后一个像素为1，相反为0，生成哈希
    for i in range(8):
        for j in range(8):
            if image[i,j]>image[i,j+1]:
                hash.append(1)
            else:
                hash.append(0)
    return hash

In [12]:
#计算汉明距离
def Hamming_distance(hash1,hash2): 
    num = 0
    for index in range(len(hash1)): 
        if hash1[index] != hash2[index]: 
            num += 1
    return num

In [22]:
image_file1 = 'image/test1.jpg'
image_file2 = 'image/test3.jpg'
img1 = cv2.imread(image_file1)
img2 = cv2.imread(image_file2)
hash1 = pHash(img1)
hash2 = pHash(img2)
dist = Hamming_distance(hash1, hash2)
#将距离转化为相似度
similarity = 1 - dist * 1.0 / 64 
print(dist)
print(similarity)

18
0.71875


In [25]:
# 下面是使用感知哈希算法比较两张图像相似程度的示例代码，基于Python和PyTorch实现。

import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

# 定义感知哈希算法实现函数
def perceptual_hash(image_path):
    # 加载图像并转换为灰度图
    img = Image.open(image_path).convert('L')
    # 缩放图像为32x32
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])
    img = transform(img)
    # 计算DCT系数
    dct_img = torch.fft.fftn(img)
    dct_img = torch.abs(dct_img[:8, :8])
    # 计算均值并生成哈希值
    mean_val = torch.mean(dct_img)
    hash_val = torch.where(dct_img > mean_val, 1, 0)
    # 将哈希值转换为整数
    hash_val = hash_val.view(-1).numpy()
    hash_int = int(''.join([str(x) for x in hash_val]), 2)
    return hash_int

# 定义计算相似度的函数
def similarity(image_path1, image_path2):
    hash1 = perceptual_hash(image_path1)
    hash2 = perceptual_hash(image_path2)
    # 计算汉明距离
    hamming_distance = bin(hash1 ^ hash2).count('1')
    # 计算相似度
    similarity = 1 - hamming_distance / 64
    return similarity

# 测试示例
if __name__ == '__main__':
    image_path1 = 'image/test1.jpg'
    image_path2 = 'image/test4.jpg'
    similarity_score = similarity(image_path1, image_path2)
    print('Similarity score:', similarity_score)

# 上述代码中，perceptual_hash函数实现了感知哈希算法，输入为图像路径，输出为哈希值。similarity函数计算了两张图像的相似度，输入为两张图像的路径，输出为相似度得分。
# 在测试示例中，我们计算了两张图像的相似度得分，并输出结果。
# 需要注意的是，这里使用的感知哈希算法实现方式可能与其他实现略有不同，因此得到的哈希值和相似度得分可能与其他实现有所不同。但是，这个示例代码可以作为感知哈希算法实现的参考，供参考和学习。

Similarity score: 0.453125
